# Teleagriculture API Tests

This notebook tests API endpoints to find the board with the most data points.

In [25]:
# Import required libraries
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict, Optional
from datetime import datetime

## API Configuration

Based on the teleagriculture project documentation, these are IoT hardware boards that send data to cloud platforms. This notebook demonstrates how to query a data platform that collects data from multiple teleagriculture boards.

In [26]:
# API Configuration for Teleagriculture Kits API (minimal)
BASE_URL = "https://kits.teleagriculture.org/api"  # official kits API base

# Optional: put KIT_API_KEY in env to POST; GETs are public per docs (but docs also mention bearer header; we support both)
import os
KIT_API_KEY = os.getenv("KIT_API_KEY")

HEADERS = {
    "Accept": "application/json",
}
if KIT_API_KEY:
    HEADERS["Authorization"] = f"Bearer {KIT_API_KEY}"

print("API:", BASE_URL)
print("Auth:", "Bearer set" if "Authorization" in HEADERS else "none")

API: https://kits.teleagriculture.org/api
Auth: none


In [27]:
# Minimal helpers per official docs
from typing import Tuple, Optional

def get_kit_info(kit_id: int) -> Optional[dict]:
    url = f"{BASE_URL}/kits/{kit_id}"
    try:
        r = requests.get(url, headers=HEADERS, timeout=30)
        if r.status_code == 200:
            return r.json().get("data")
        return None
    except requests.RequestException:
        return None


def count_sensor_measurements(kit_id: int, sensor_name: str, page_size: int = 50, max_pages: int = 200) -> int:
    """Count all measurements for a kit sensor using cursor pagination.
    Limits pages to avoid unbounded runs.
    """
    total = 0
    cursor = None
    pages = 0
    while pages < max_pages:
        params = {"page[size]": str(page_size)}
        if cursor:
            params["page[cursor]"] = cursor
        url = f"{BASE_URL}/kits/{kit_id}/{sensor_name}/measurements"
        try:
            r = requests.get(url, headers=HEADERS, params=params, timeout=30)
        except requests.RequestException:
            break
        if r.status_code == 404:
            break
        if r.status_code != 200:
            break
        try:
            body = r.json()
        except Exception:
            break
        data = body.get("data")
        if isinstance(data, list):
            total += len(data)
        else:
            break
        meta = body.get("meta", {})
        cursor = meta.get("next_cursor")
        pages += 1
        if not cursor:
            break
    return total

## Fetch Boards Function

Function to retrieve all registered teleagriculture boards from the data platform API. Each "board" represents a deployed IoT device collecting agricultural data.

## Fetch all sensors for a kit and count in parallel

Minimal helpers to grab all sensors from one kit and count each sensor’s datapoints concurrently.

In [28]:
from concurrent.futures import ThreadPoolExecutor, as_completed


def get_kit_sensors(kit_id: int) -> list[dict]:
    kit = get_kit_info(kit_id)
    if not kit:
        return []
    sensors = kit.get("sensors") or []
    # normalize: keep only id and name if present
    out = []
    for s in sensors:
        if isinstance(s, dict) and s.get("name"):
            out.append({"id": s.get("id"), "name": s.get("name")})
    return out


def count_all_sensors_for_kit(kit_id: int, page_size: int = 50, max_workers: int = 8) -> dict:
    sensors = get_kit_sensors(kit_id)
    if not sensors:
        return {"kit_id": kit_id, "counts": {}, "best": None}

    counts: dict[str, int] = {}
    best = {"sensor": None, "count": -1}

    def _worker(sname: str) -> tuple[str, int]:
        c = count_sensor_measurements(kit_id, sname, page_size=page_size)
        return sname, c

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = {ex.submit(_worker, s["name"]): s["name"] for s in sensors}
        for fut in as_completed(futures):
            sname = futures[fut]
            try:
                sname, c = fut.result()
                counts[sname] = c
                if c > best["count"]:
                    best = {"sensor": sname, "count": c}
            except Exception:
                counts[sname] = 0
    return {"kit_id": kit_id, "counts": counts, "best": best}

# minimal run example (change the kit id here)
one_kit_result = count_all_sensors_for_kit(1001, page_size=50)
print("KIT", one_kit_result["kit_id"], "BEST", one_kit_result["best"])

KIT 1001 BEST {'sensor': 'NH3', 'count': 1221}


In [29]:
def fetch_all_boards() -> List[Dict]:
    """
    Fetch all registered teleagriculture boards from the data platform.
    
    Returns:
        List[Dict]: List of board objects with metadata, or empty list if error occurs
    """
    try:
        # Common API endpoints for IoT platforms that might host teleagriculture data
        possible_endpoints = [
            "/devices",           # Common IoT platform endpoint
            "/boards",           # Board-specific endpoint
            "/nodes",            # LoRaWAN nodes
            "/sensors",          # Sensor networks
            "/stations"          # Weather/agri stations
        ]
        
        for endpoint in possible_endpoints:
            try:
                url = f"{BASE_URL}{endpoint}"
                response = requests.get(url, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    data = response.json()
                    
                    # Handle different response formats
                    if isinstance(data, list):
                        boards = data
                    elif isinstance(data, dict):
                        # Try common keys for device arrays
                        for key in ['devices', 'boards', 'nodes', 'sensors', 'stations', 'data', 'results']:
                            if key in data and isinstance(data[key], list):
                                boards = data[key]
                                break
                        else:
                            boards = []
                    else:
                        boards = []
                    
                    if boards:
                        print(f"✅ Successfully fetched {len(boards)} boards from {endpoint}")
                        return boards
                        
            except Exception as e:
                continue  # Try next endpoint
                
        print("❌ Could not find boards at any common endpoint")
        return []
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"❌ JSON decode error: {e}")
        return []
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return []

# Test the function (will be used later)
print("📡 Board fetching function defined successfully!")
print("🌿 Ready to query teleagriculture board data from platform API.")

📡 Board fetching function defined successfully!
🌿 Ready to query teleagriculture board data from platform API.


## Data Point Counting Function

Function to count sensor data points collected by each teleagriculture board. This could include temperature readings, soil moisture, humidity, light levels, etc.

In [30]:
def count_board_data_points(board_id: str) -> int:
    """
    Count sensor data points collected by a specific teleagriculture board.
    
    Args:
        board_id (str): The ID of the teleagriculture board
        
    Returns:
        int: Number of data points (sensor readings) collected by the board
    """
    try:
        # Teleagriculture boards typically send sensor data to these types of endpoints
        possible_endpoints = [
            f"/devices/{board_id}/data",        # Device data endpoint
            f"/devices/{board_id}/measurements", # Measurement endpoint  
            f"/devices/{board_id}/sensors",     # Sensor readings
            f"/boards/{board_id}/readings",     # Board readings
            f"/nodes/{board_id}/uplinks",       # LoRaWAN uplink messages
            f"/stations/{board_id}/observations" # Weather station observations
        ]
        
        for endpoint in possible_endpoints:
            try:
                url = f"{BASE_URL}{endpoint}"
                response = requests.get(url, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    data = response.json()
                    
                    # Handle different data formats from IoT platforms
                    if isinstance(data, list):
                        count = len(data)
                    elif isinstance(data, dict):
                        # Try common keys for sensor data arrays
                        for key in ['measurements', 'readings', 'data', 'sensors', 'uplinks', 'observations', 'records']:
                            if key in data and isinstance(data[key], list):
                                count = len(data[key])
                                break
                        else:
                            # Count sensor types if structured differently
                            sensor_keys = ['temperature', 'humidity', 'pressure', 'soil_moisture', 'light', 'ph', 'nitrogen']
                            count = sum(1 for key in sensor_keys if key in data and data[key] is not None)
                            
                            if count == 0:
                                count = len(data)  # Fallback to total keys
                    else:
                        count = 0
                    
                    print(f"📊 Board {board_id}: {count} data points found via {endpoint}")
                    return count
                    
            except Exception as e:
                continue  # Try next endpoint
                
        print(f"⚠️  Could not fetch sensor data for board {board_id}")
        return 0
        
    except Exception as e:
        print(f"❌ Error counting data points for board {board_id}: {e}")
        return 0

def get_board_data_counts(boards: List[Dict]) -> Dict[str, Dict]:
    """
    Get sensor data counts for all teleagriculture boards.
    
    Args:
        boards (List[Dict]): List of board/device objects from the platform
        
    Returns:
        Dict[str, Dict]: Dictionary with board info and data counts
    """
    board_stats = {}
    
    for board in boards:
        # Handle different IoT platform object structures
        board_id = (board.get('id') or board.get('device_id') or board.get('node_id') or 
                   board.get('sensor_id') or board.get('station_id') or board.get('_id'))
        
        board_name = (board.get('name') or board.get('device_name') or board.get('label') or 
                     board.get('title') or board.get('station_name') or f"Board {board_id}")
        
        # Get location info if available (common in agricultural IoT)
        location = board.get('location') or board.get('coordinates') or board.get('position')
        
        if board_id:
            data_count = count_board_data_points(str(board_id))
            board_stats[board_id] = {
                'name': board_name,
                'data_count': data_count,
                'location': location,
                'board_info': board
            }
        else:
            print(f"⚠️  Skipping board without ID: {board}")
    
    return board_stats

print("📡 Sensor data counting functions defined successfully!")
print("🌱 Ready to analyze agricultural sensor data from teleagriculture boards.")

📡 Sensor data counting functions defined successfully!
🌱 Ready to analyze agricultural sensor data from teleagriculture boards.


## Find Board with Most Data Points

Main execution logic to analyze all boards and identify the one with the most data points.

In [31]:
def find_board_with_most_data():
    """
    Main function to find the board with the most data points.
    """
    print("🚀 Starting board analysis...")
    print("=" * 50)
    
    # Step 1: Fetch all boards
    print("📋 Fetching all boards...")
    boards = fetch_all_boards()
    
    if not boards:
        print("❌ No boards found or error occurred. Check your API configuration.")
        return None
    
    print(f"✅ Found {len(boards)} boards")
    print()
    
    # Step 2: Count data points for each board
    print("📊 Counting data points for each board...")
    board_stats = get_board_data_counts(boards)
    
    if not board_stats:
        print("❌ Could not get data counts for any boards.")
        return None
    
    # Step 3: Find the board with the most data points
    max_board_id = max(board_stats.keys(), key=lambda k: board_stats[k]['data_count'])
    max_board = board_stats[max_board_id]
    
    print()
    print("🏆 RESULTS")
    print("=" * 50)
    print(f"Board with most data points:")
    print(f"  📋 Name: {max_board['name']}")
    print(f"  🆔 ID: {max_board_id}")
    print(f"  📊 Data Points: {max_board['data_count']}")
    print()
    
    # Summary of all boards
    print("📋 All Boards Summary:")
    print("-" * 30)
    sorted_boards = sorted(board_stats.items(), key=lambda x: x[1]['data_count'], reverse=True)
    
    for i, (board_id, stats) in enumerate(sorted_boards, 1):
        emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "📋"
        print(f"{emoji} {stats['name']}: {stats['data_count']} data points")
    
    return {
        'winner': max_board,
        'winner_id': max_board_id,
        'all_stats': board_stats
    }

# Execute the analysis
result = find_board_with_most_data()

🚀 Starting board analysis...
📋 Fetching all boards...
❌ Could not find boards at any common endpoint
❌ No boards found or error occurred. Check your API configuration.
❌ Could not find boards at any common endpoint
❌ No boards found or error occurred. Check your API configuration.


## Data Visualization

Create charts and detailed analysis of the board data points.

## Sensor Scan: IDs 1001–1060

Iterate over sensor IDs 1001 to 1060, query the platform API, and find the sensor with the most datapoints.

In [32]:
# Minimal scan: kits 1001..1060 — find sensor with most datapoints

def find_max_sensor_in_range(start_kit: int = 1015, end_kit: int = 1060, page_size: int = 50) -> dict:
    best = {"kit_id": None, "sensor": None, "count": -1}
    for kit_id in range(start_kit, end_kit + 1):
        kit = get_kit_info(kit_id)
        if not kit or not isinstance(kit, dict):
            print(f"kit {kit_id}: not found")
            continue
        sensors = kit.get("sensors") or []
        if not sensors:
            print(f"kit {kit_id}: no sensors")
            continue
        for s in sensors:
            name = s.get("name")
            if not name:
                continue
            cnt = count_sensor_measurements(kit_id, name, page_size=page_size)
            print(f"kit {kit_id} sensor {name}: {cnt}")
            if cnt > best["count"]:
                best = {"kit_id": kit_id, "sensor": name, "count": cnt}
    return best

best = find_max_sensor_in_range(1001, 1060, page_size=50)
print("\nRESULT")
print(best)

kit 1001 sensor ftTemp: 1219
kit 1001 sensor gbHum: 1219
kit 1001 sensor gbHum: 1219
kit 1001 sensor gbTemp: 1219
kit 1001 sensor gbTemp: 1219
kit 1001 sensor Moist: 1219
kit 1001 sensor Moist: 1219
kit 1001 sensor CO: 1221
kit 1001 sensor CO: 1221
kit 1001 sensor NO2: 1221
kit 1001 sensor NO2: 1221
kit 1001 sensor NH3: 1221
kit 1001 sensor NH3: 1221
kit 1001 sensor C3H8: 1221
kit 1001 sensor C3H8: 1221
kit 1001 sensor C4H10: 1221
kit 1001 sensor C4H10: 1221
kit 1001 sensor CH4: 1221
kit 1001 sensor CH4: 1221
kit 1001 sensor H2: 1221
kit 1001 sensor H2: 1221
kit 1001 sensor C2H5OH: 1221
kit 1001 sensor pH: 1219
kit 1001 sensor NO3: 0
kit 1001 sensor NO2_aq: 0
kit 1001 sensor GH: 0
kit 1001 sensor KH: 0
kit 1001 sensor pH_strip: 0
kit 1001 sensor Cl2: 0
kit 1002 sensor ftTemp: 1218
kit 1002 sensor gbHum: 1218
kit 1002 sensor gbTemp: 1218
kit 1002 sensor Moist: 1218
kit 1002 sensor CO: 1218
kit 1002 sensor NO2: 1218
kit 1002 sensor NH3: 1218
kit 1002 sensor C3H8: 1218
kit 1002 sensor C4H

KeyboardInterrupt: 

In [18]:
from collections import defaultdict

def _count_datapoints_from_response(data) -> int:
    """Best-effort count of datapoints from arbitrary API responses."""
    if data is None:
        return 0
    if isinstance(data, list):
        return len(data)
    if isinstance(data, dict):
        # Prefer common array keys
        for key in [
            'data', 'results', 'measurements', 'readings', 'entries', 'values',
            'observations', 'records', 'points'
        ]:
            if key in data and isinstance(data[key], list):
                return len(data[key])
        # Fallback: count scalar series
        return sum(1 for v in data.values() if isinstance(v, (int, float, str, bool)))
    return 0


def fetch_sensor_datapoints(sensor_id: int) -> tuple[int, dict]:
    """
    Try multiple likely endpoints for a sensor and return the datapoint count and last successful meta.
    Returns (count, meta) where meta contains endpoint and status.
    """
    endpoints = [
        f"/sensors/{sensor_id}",
        f"/sensors/{sensor_id}/data",
        f"/sensors/{sensor_id}/readings",
        f"/sensors/{sensor_id}/measurements",
        f"/devices/{sensor_id}/data",
        f"/nodes/{sensor_id}/uplinks",
    ]

    last_error = None
    for ep in endpoints:
        url = f"{BASE_URL.rstrip('/')}{ep}"
        try:
            r = requests.get(url, headers=HEADERS, timeout=30)
            if r.status_code == 200:
                try:
                    data = r.json()
                except Exception:
                    data = None
                count = _count_datapoints_from_response(data)
                return count, {"endpoint": ep, "status": r.status_code}
            else:
                last_error = {"endpoint": ep, "status": r.status_code, "text": r.text[:200]}
        except requests.RequestException as e:
            last_error = {"endpoint": ep, "error": str(e)}
            continue
    return 0, (last_error or {"endpoint": None, "error": "no-endpoint-succeeded"})


def scan_sensors_and_find_max(start_id: int = 1001, end_id: int = 1060):
    print(f"🔎 Scanning sensors from {start_id} to {end_id}...")
    best = {
        "sensor_id": None,
        "count": -1,
        "meta": {}
    }
    results = {}

    for sid in range(start_id, end_id + 1):
        count, meta = fetch_sensor_datapoints(sid)
        results[sid] = {"count": count, "meta": meta}
        print(f"Sensor {sid}: {count} datapoints (via {meta.get('endpoint')})")
        if count > best["count"]:
            best = {"sensor_id": sid, "count": count, "meta": meta}

    print("\n🏁 Scan complete.")
    if best["sensor_id"] is not None:
        print("🏆 Sensor with most datapoints:")
        print(f"  🆔 ID: {best['sensor_id']}")
        print(f"  📊 Count: {best['count']}")
        print(f"  🔗 Endpoint: {best['meta'].get('endpoint')}")
    else:
        print("No sensors returned datapoints in the given range.")

    return {"best": best, "results": results}

# Run the scan now
scan_result = scan_sensors_and_find_max(1001, 1060)

🔎 Scanning sensors from 1001 to 1060...
Sensor 1001: 0 datapoints (via /sensors/1001)
Sensor 1002: 0 datapoints (via /sensors/1002)
Sensor 1003: 0 datapoints (via /sensors/1003)
Sensor 1004: 0 datapoints (via /sensors/1004)
Sensor 1005: 0 datapoints (via /sensors/1005)
Sensor 1006: 0 datapoints (via /sensors/1006)
Sensor 1007: 0 datapoints (via /sensors/1007)
Sensor 1008: 0 datapoints (via /sensors/1008)
Sensor 1009: 0 datapoints (via /sensors/1009)
Sensor 1010: 0 datapoints (via /sensors/1010)
Sensor 1011: 0 datapoints (via /sensors/1011)
Sensor 1012: 0 datapoints (via /sensors/1012)
Sensor 1013: 0 datapoints (via /sensors/1013)
Sensor 1014: 0 datapoints (via /sensors/1014)
Sensor 1015: 0 datapoints (via /sensors/1015)
Sensor 1016: 0 datapoints (via /sensors/1016)
Sensor 1017: 0 datapoints (via /sensors/1017)
Sensor 1018: 0 datapoints (via /sensors/1018)
Sensor 1019: 0 datapoints (via /sensors/1019)
Sensor 1020: 0 datapoints (via /sensors/1020)
Sensor 1021: 0 datapoints (via /sensors/

In [16]:
def create_board_analysis_chart(board_stats: Dict[str, Dict]):
    """
    Create visualizations for board data analysis.
    
    Args:
        board_stats (Dict[str, Dict]): Board statistics from get_board_data_counts
    """
    if not board_stats:
        print("❌ No board statistics available for visualization.")
        return
    
    # Prepare data for plotting
    board_names = [stats['name'] for stats in board_stats.values()]
    data_counts = [stats['data_count'] for stats in board_stats.values()]
    board_ids = list(board_stats.keys())
    
    # Create DataFrame for better handling
    df = pd.DataFrame({
        'Board ID': board_ids,
        'Board Name': board_names,
        'Data Points': data_counts
    })
    
    # Sort by data points for better visualization
    df = df.sort_values('Data Points', ascending=True)
    
    # Create the plot
    plt.figure(figsize=(12, 8))
    
    # Horizontal bar chart
    bars = plt.barh(range(len(df)), df['Data Points'], color='skyblue', alpha=0.7)
    
    # Customize the plot
    plt.yticks(range(len(df)), df['Board Name'])
    plt.xlabel('Number of Data Points')
    plt.title('Data Points per Board - Teleagriculture API Analysis', fontsize=16, fontweight='bold')
    plt.grid(axis='x', alpha=0.3)
    
    # Add value labels on bars
    for i, (bar, value) in enumerate(zip(bars, df['Data Points'])):
        plt.text(value + max(df['Data Points']) * 0.01, bar.get_y() + bar.get_height()/2, 
                str(value), va='center', fontweight='bold')
    
    # Highlight the board with most data points
    max_idx = df['Data Points'].idxmax()
    bars[df.index.get_loc(max_idx)].set_color('gold')
    bars[df.index.get_loc(max_idx)].set_alpha(1.0)
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed statistics
    print("📊 DETAILED STATISTICS")
    print("=" * 50)
    print(f"Total boards analyzed: {len(df)}")
    print(f"Total data points across all boards: {df['Data Points'].sum()}")
    print(f"Average data points per board: {df['Data Points'].mean():.1f}")
    print(f"Median data points per board: {df['Data Points'].median():.1f}")
    print(f"Standard deviation: {df['Data Points'].std():.1f}")
    print()
    
    # Show top 3 boards
    top_3 = df.nlargest(3, 'Data Points')
    print("🏆 TOP 3 BOARDS:")
    for i, (_, row) in enumerate(top_3.iterrows(), 1):
        emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
        print(f"{emoji} {row['Board Name']}: {row['Data Points']} data points")
    
    return df

# Create visualization if we have results
if 'result' in locals() and result and result.get('all_stats'):
    print("📈 Creating visualization...")
    df_analysis = create_board_analysis_chart(result['all_stats'])
else:
    print("⚠️  Run the board analysis first to see visualizations.")
    print("💡 Make sure to update the API_KEY and BASE_URL in the configuration section.")

⚠️  Run the board analysis first to see visualizations.
💡 Make sure to update the API_KEY and BASE_URL in the configuration section.


## Simple helper: get all sensor data for a kit id

This function fetches all available measurements for a given kit (board) id across all its sensors and returns a tidy pandas DataFrame. It uses the same BASE_URL and HEADERS configured above and follows the API's cursor pagination automatically.

- Input: kit_id (int)
- Optional: sensors (list[str]) to limit which sensors to fetch; defaults to all sensors on the kit
- Output: pandas DataFrame with columns like: kit_id, sensor, timestamp/value/..., depending on the API payload

In [ ]:
from typing import Iterable, Any

def _paginate(url: str, params: dict | None = None, headers: dict | None = None, page_size: int = 100, max_pages: int = 500):
    """Generator yielding pages from cursor-paginated endpoint returning {'data': [...], 'meta': {'next_cursor': '...'}}"""
    params = dict(params or {})
    params["page[size]"] = str(page_size)
    cursor = None
    pages = 0
    while pages < max_pages:
        if cursor:
            params["page[cursor]"] = cursor
        try:
            r = requests.get(url, headers=headers, params=params, timeout=30)
        except requests.RequestException:
            break
        if r.status_code != 200:
            break
        try:
            payload = r.json()
        except Exception:
            break
        data = payload.get("data")
        meta = payload.get("meta", {})
        yield data if isinstance(data, list) else []
        cursor = meta.get("next_cursor")
        pages += 1
        if not cursor:
            break


def get_kit_measurements_df(kit_id: int, sensors: Iterable[str] | None = None, page_size: int = 100) -> pd.DataFrame:
    """
    Fetch all measurements for a given kit across selected sensors and return a tidy DataFrame.

    - kit_id: numeric id of the kit/board
    - sensors: optional list of sensor names; if None, will discover sensors via get_kit_info(kit_id)
    - page_size: page size for cursor pagination

    Returns a DataFrame with columns: kit_id, sensor, timestamp, value, unit, _raw
    (Columns may include NaNs if the API doesn't provide those fields.)
    """
    # Discover sensors if not provided
    sensor_list: list[str]
    if sensors is None:
        kit = get_kit_info(kit_id)
        if not kit:
            return pd.DataFrame(columns=["kit_id", "sensor", "timestamp", "value", "unit", "_raw"])
        sensor_list = [s.get("name") for s in (kit.get("sensors") or []) if isinstance(s, dict) and s.get("name")]
    else:
        sensor_list = [s for s in sensors if s]

    rows: list[dict[str, Any]] = []

    for sname in sensor_list:
        base = f"{BASE_URL}/kits/{kit_id}/{sname}/measurements"
        for page in _paginate(base, headers=HEADERS, page_size=page_size):
            for item in page:
                if not isinstance(item, dict):
                    continue
                rec = item
                # Some APIs wrap fields inside 'attributes'
                if isinstance(rec.get("attributes"), dict):
                    # merge attributes shallowly (attributes wins for overlapping keys)
                    rec = {**{k: v for k, v in rec.items() if k != "attributes"}, **rec["attributes"]}
                # Normalize common fields
                ts = rec.get("timestamp") or rec.get("time") or rec.get("created_at") or rec.get("datetime")
                val = rec.get("value") or rec.get("reading") or rec.get("measurement") or rec.get("val")
                unit = rec.get("unit") or rec.get("units")
                rows.append({
                    "kit_id": kit_id,
                    "sensor": sname,
                    "timestamp": ts,
                    "value": val,
                    "unit": unit,
                    "_raw": item,  # keep original
                })

    df = pd.DataFrame(rows)
    # Coerce timestamp and sort
    if not df.empty and "timestamp" in df.columns:
        try:
            df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=True)
            df = df.sort_values(["sensor", "timestamp"], kind="stable")
        except Exception:
            pass
    return df

In [ ]:
# Demo: fetch all data for a kit id (adjust kit_id)
KIT_DEMO_ID = 1001  # change as needed

df_all = get_kit_measurements_df(KIT_DEMO_ID)
print(f"Fetched {len(df_all)} rows for kit {KIT_DEMO_ID}")
df_all.head()

In [ ]:
# Simplest helper: get a DataFrame for a kit id

def get_kit_df(kit_id: int) -> pd.DataFrame:
    return get_kit_measurements_df(kit_id)

# Example usage:
# df = get_kit_df(1001)
# df.head()